# The ResNet architecture, from identity block to the whole structure

* The first part of building the ResNet architecture is the identity block,

Which will be the base for the whole structure.

In [ ]:
#!/usr/bin/env python3
"""
Makes the ResNet model identity block.
"""


import tensorflow.keras as keras


def identity_block(A_prev, filters):
    """
    Identity block function.
    
    :param A_prev: output of the previous layer.
    
    :param filters: number of filters in tuple form carrying:
    F11 - filter for 1x1 convolution
    F3 - filter size for 3x3 convolution
    F22 - filter for second 1z1 convolution
    
    All convolutions will be followed by Batch normalization, and ReLU activation.
    
    All weights will use normal initialization.
    
    Returns: Activated output of the block.
    """
    F11, F3, F12 = filters
    initializer = keras.initializers.he_normal()
    activation = keras.activations.relu
    
    layers = keras.layers
    
    Conv_1x1 = layers.Conv2D(
        F11,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,   
    )(A_prev)
    Batch_1x1 = layers.BatchNormalization(axis=3)(Conv_1x1)
    ReLU_1x1 = layers.Activation(activation)(Batch_1x1)
    
    Conv_3x3 = layers.Conv2D(
        F3,
        (3, 3),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_1x1)
    Batch_3x3 = layers.BatchNormalization(axis=3)(Conv_3x3)
    ReLU_3x3 = layers.Activation(activation)(Batch_3x3)
    
    Conv_1x1_2 = layers.Conv2D(
        F12,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_3x3)
    Batch_1x1_2 = layers.BatchNormalization(axis=3)(Conv_1x1_2)
    
    pre_output = layers.Add()([Batch_1x1_2, A_prev])
    
    output = layers.Activation(activation)(pre_output)
    
    return output


In [ ]:
# Main function for Identity Block

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 256))
    Y = identity_block(X, [64, 64, 256])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

# Resnet Projection Block
After Completion of the identity_block, we can move on to the projection_block.

The projection block is used when the input and output dimensions are different for eachother.
* The way it works is this:

> It includes a convolutional layer with a stride of (2, 2), in order to downsample the input
> 
> And a convolutional layer with a (1, 1) filter size to change the depth of the input to match the output. 

In [ ]:
#!/usr/bin/env python3
"""
Makes the Projection blocks of the ResNet Architecture
"""


import tensorflow.keras as keras


def projection_block(A_prev, filters, s=2):
    """
    Projection block function.
    
    :param A_prev: output of the previous layer.
    
    :param filters: number of filters in tuple form carrying:
    F11 - filter for 1x1 convolution
    F3 - filter size for 3x3 convolution
    F12 -filter for second 1z1 convolution as well as the shortcut convolution
    
    s- stride of the main and shortcut convolution
    
    All convolutions will be followed by Batch normalization,
    and ReLU activation.
    
    All weights will use he_normal initialization.
    Seed of he_normal initialization will be set to 0
    
    Returns: Activated output of the block.
    """
    F11, F3, F12 = filters
    initializer = keras.initializers.he_normal(seed=0)
    activation = keras.activations.relu
    layers = keras.layers
    batch = layers.BatchNormalization
    
    Conv_1x1 = layers.Conv2D(
        F11,
        (1, 1),
        strides=s,
        padding='same',
        kernel_initializer=initializer,
    )(A_prev)
    Batch_1x1 = batch(axis=3)(Conv_1x1)
    ReLU_1x1 = layers.Activation(activation)(Batch_1x1)
    
    Conv_3x3 = layers.Conv2D(
        F3,
        (3, 3),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_1x1)
    Batch_3x3 = batch(axis=3)(Conv_3x3)
    ReLU_3x3 = layers.Activation(activation)(Batch_3x3)
    
    Conv_1x1_2 = layers.Conv2D(
        F12,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_3x3)
    Batch_1x1_2 = batch(axis=3)(Conv_1x1_2)
    
    skip_layer = layers.Conv2D(
        F12,
        (1, 1),
        strides=s,
        padding='same',
        kernel_initializer=initializer,
    )(A_prev)
    skip_Batch = batch(axis=3)(skip_layer)
    
    pre_output = layers.Add()([Batch_1x1_2, skip_Batch])
    
    output = layers.Activation(activation)(pre_output)
    
    return output


In [ ]:
# Main function of Projection Block

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 3))
    Y = projection_block(X, [64, 64, 256])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

# With the Founding Blocks complete, All that is left is the model

* Using the modules built we will make the ResNet Architecture.

In [ ]:
#!/usr/bin/env python3
"""
Makes the ResNet Architecture
"""

from tensorflow import keras
"""
identity_block = __import__('2-identity_block').identity_block
projection_block = __import__('3-projection_block').projection_block
"""


def resnet50():
    """
    ResNet50 architecture.
    
    The input image is 224x224 RGB.
    
    All convolutions will be followed by Batch normalization,
    and ReLU activation.
    
    All weights will use he_normal initialization.
    Seed of he_normal initialization will be set to 0
    
    Returns: Keras Model
    """
    init = keras.initializers.he_normal(seed=0)
    activation = keras.activations.relu
    input = keras.Input(shape=(224, 224, 3))
    layers = keras.layers
    
    start = layers.Conv2D(
        64,
        (7, 7),
        strides=(2, 2),
        kernel_initializer=init,
    )(input)
    start_batch = layers.BatchNormalization(axis=3)(start)
    start_relu = layers.Activation(activation)(start_batch)
    
    max_pool_1 = (layers.MaxPool2D(
        pool_size=(3, 3),
        strides=(2, 2),
        padding='same',
    )(start_relu))
    
    project_1 = projection_block(start_relu, [128, 128, 512], s=1)
    identity_1 = identity_block(
        project_1,
        [64, 64, 256],
    )
    identity_2 = identity_block(
        identity_1,
        [64, 64, 256],
    )
    
    project_2 = projection_block(identity_2, [64, 64, 256], s=2)
    identity_2_1 = identity_block(
        project_2,
        [128, 128, 512]
    )
    identity_2_2 = identity_block(
        identity_2_1,
        [128, 128, 512],
    )
    identity_2_3 = identity_block(
        identity_2_2,
        [128, 128, 512],
    )
    
    project_3 = projection_block(identity_2_3, [256, 256, 1024], s=2)
    identity_3_1 = identity_block(
        project_3,
        [256, 256, 1024],
    )
    identity_3_2 = identity_block(
        identity_3_1,
        [256, 256, 1024],
    )
    identity_3_3 = identity_block(
        identity_3_2,
        [256, 256, 1024],
    )
    identity_3_4 = identity_block(
        identity_3_3,
        [256, 256, 1024],
    )
    identity_3_5 = identity_block(
        identity_3_4,
        [256, 256, 1024],
    )
    
    project_4 = projection_block(identity_3_5, [512, 512, 2048], s=2)
    identity_4_1 = identity_block(
        project_4,
        [512, 512, 2048],
    )
    identity_4_2 = identity_block(
        identity_4_1,
        [512, 512, 2048],
    )
    
    average_pooling = layers.AveragePooling2D(
        pool_size=(7, 7),
        strides=(1, 1),
        padding="valid"
    )(identity_4_2)
    
    output = layers.Dense(
        1000,
        activation="softmax",
        kernel_initializer=init,
    )(average_pooling)
    
    model = keras.models.Model(inputs=input, outputs=output)
    
    return model
    

In [11]:
# Main function for Resnet50
#!/usr/bin/env python3
"""
makes the Google inception block
"""


import tensorflow.keras as keras


def inception_block(A_prev, filters):
    """
    Makes the Google inception block

    :param A_prev: input tensor from previous layer

    :param filters: filters is a tuple or list containing
    F1, F3R, F3,F5R, F5, FPP, respectively

    all conv layers must use ReLU activation function

    Returns: the concatenated output of the inception block
"""
    layers = keras.layers
    f1 = filters[0]
    f3r = filters[1]
    f3 = filters[2]
    f5r = filters[3]
    f5 = filters[4]
    fpp = filters[5]

    tower_1 = layers.Conv2D(
        filters=f1,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    tower_2 = layers.Conv2D(
        filters=f3r,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    tower_3 = layers.Conv2D(
        filters=f3,
        kernel_size=(3, 3),
        padding='same',
        activation='relu',
    )

    tower_4 = layers.Conv2D(
        filters=f5r,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    tower_5 = layers.Conv2D(
        filters=f5,
        kernel_size=(5, 5),
        padding='same',
        activation='relu',
    )

    pool = layers.MaxPooling2D((3, 3), strides=(1, 1),
                               padding='same')(A_prev)

    tower_6 = layers.Conv2D(
        filters=fpp,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    layer_1 = tower_1(A_prev)
    layer_3_prev = tower_2(A_prev)
    layer_3 = tower_3(layer_3_prev)
    layer_5_prev = tower_4(A_prev)
    layer_5 = tower_5(layer_5_prev)
    layer_end = tower_6(pool)

    output = layers.concatenate([layer_1, layer_3, layer_5, layer_end])

    return output

#!/usr/bin/env python3



if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 224, 224,  │        384 │ input_layer_4[0]… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 224, 224,  │         64 │ input_layer_4[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (MaxPooling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 224, 224,  │        256 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 224, 224,  │    110,720 │ conv2d_8[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 224, 224,  │     12,832 │ conv2d_10[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 224, 224,  │        128 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 224, 224,  │          0 │ conv2d_7[0][0],   │
│ (Concatenate)       │ 256)              │            │ conv2d_9[0][0],   │
│                     │                   │            │ conv2d_11[0][0],  │
│                     │                   │            │ conv2d_12[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,384 (485.88 KB)

 Trainable params: 124,384 (485.88 KB)

 Non-trainable params: 0 (0.00 B)